In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
heights = np.arange(60, 78, .1)

In [ ]:
heights

In [ ]:
heights.size

In [ ]:
np.random.seed(0)
random_fluctuation = np.random.normal(scale=10, size=heights.size)
weights = 4 * heights - 130 + random_fluctuation

In [ ]:
measurements = np.array([heights, weights]).T

In [ ]:
measurements

In [ ]:
plt.scatter(measurements[:, 0], measurements[:, 1])
plt.xlabel('Height (in)')
plt.ylabel('Weight (lb)')

In [ ]:
plt.scatter(measurements[:, 0], measurements[:, 1])
plt.xlabel('Height (in)')
plt.ylabel('Weight (lb)')
plt.axis('equal')

In [ ]:
centered_data = np.array([heights - heights.mean(), weights - weights.mean()])

In [ ]:
centered_data.shape

In [ ]:
plt.scatter(centered_data[0], centered_data[1])
plt.axhline(0, c='black')
plt.axvline(0, c='black')
plt.xlabel('Centered Height (in)')
plt.ylabel('Centered Weight (lb)')
plt.axis('equal')

In [ ]:
from math import sin, cos
angle = np.radians(-90)

In [ ]:
rotation_matrix = np.array([[cos(angle), -sin(angle)], [sin(angle), cos(angle)]])

In [ ]:
rotated_data = rotation_matrix @ centered_data

In [ ]:
plt.scatter(centered_data[0], centered_data[1], label='Original Data')
plt.scatter(rotated_data[0], rotated_data[1], c='y', label='Rotated Data')
plt.axhline(0, c='black')
plt.axvline(0, c='black')
plt.legend()
plt.axis('equal')

In [ ]:
data_labels = ['unrotated', 'rotated']
data_list = [centered_data, rotated_data]
for data_label, data in zip(data_labels, data_list):
    y_values = data[1]
    penalty = y_values @ y_values / y_values.size
    print(f"The penalty score for the {data_label} data is {penalty:.2f}")

In [ ]:
for data_label, data in zip(data_labels, data_list):
    y_var = data[1].var()
    penalty = data[1] @ data[1] / data[0].size
    assert round(y_var, 14) == round(penalty, 14)
    print(f"The y-axis variance for the {data_label} data is {y_var:.2f}")

In [ ]:
for data_label, data in zip(data_labels, data_list):
    x_var = data[0].var()
    print(f"The x-axis variance for the {data_label} data is {x_var:.2f}")

In [ ]:
total_variance = centered_data[0].var() + centered_data[1].var()
assert total_variance == rotated_data[0].var() + rotated_data[1].var()

In [ ]:
for data_label, data in zip(data_labels, data_list):
    percent_x_axis_var = 100 * data[0].var() / total_variance
    percent_y_axis_var = 100 * data[1].var() / total_variance
    print(f"In the {data_label} data, {percent_x_axis_var:.2f}% of the total variance is distributed across the x-axis")
    print(f"The remaining {percent_y_axis_var:.2f}% of the total variance is distributed across the y-axis\n")

In [ ]:
def rotate(angle, data=centered_data):
    angle = np.radians(-angle)
    rotation_matrix = np.array([[cos(angle), -sin(angle)], [sin(angle), cos(angle)]])
    return rotation_matrix @ data
angles = np.arange(1, 180, .1)
x_variances = [rotate(angle)[0].var() for angle in angles]
percent_x_variances = 100 * np.array(x_variances) / total_variance
optimal_index = np.argmax(percent_x_variances)
optimal_angle = angles[optimal_index]
plt.plot(angles, percent_x_variances)
plt.axvline(optimal_angle, c='k')
plt.xlabel('Angle (degrees)')
plt.ylabel('% x-axis coverage')
plt.show()
max_coverage = percent_x_variances[optimal_index]
max_x_var = x_variances[optimal_index]
print(f"The horizontal variance is maximized to approximately {int(max_x_var)} after a {optimal_angle:.1f} degree rotation.")
print(f"That rotation distributes {max_coverage:.2f}% of the total variance onto the x-axis.")

In [ ]:
best_rotated_data = rotate(optimal_angle)
plt.scatter(best_rotated_data[0], best_rotated_data[1])
plt.axhline(0, c='black')
plt.axvline(0, c='black')
plt.axis('equal')

In [ ]:
optimal_angle

In [ ]:
x_values = best_rotated_data[0]
sorted_x_values = sorted(x_values)
cluster_size = int(x_values.size / 3)
small_cutoff = max(sorted_x_values[:cluster_size])
large_cutoff = min(sorted_x_values[-cluster_size:])
print(f"A 1D threshold of {small_cutoff:.2f} separates the small-sized "
"and medium-sized customers.")
print(f"A 1D threshold of {large_cutoff:.2f} separates the medium-sized "
"and large-sized customers.")

In [ ]:
def plot_customer_segments(horizontal_2d_data):
    small, medium, large = [], [], []
    cluster_labels = ['Small', 'Medium', 'Large']
    for x_value, y_value in horizontal_2d_data.T:
        if x_value <= small_cutoff:
            small.append([x_value, y_value])
        elif small_cutoff < x_value < large_cutoff:
            medium.append([x_value, y_value])
        else:
            large.append([x_value, y_value])
    for i, cluster in enumerate([small, medium, large]):
        cluster_x_values, cluster_y_values = np.array(cluster).T
        plt.scatter(cluster_x_values, cluster_y_values, color=['g', 'b', 'y'][i], label=cluster_labels[i])
    plt.axhline(0, c='black')
    plt.axvline(large_cutoff, c='black', linewidth=3, linestyle='--')
    plt.axvline(small_cutoff, c='black', linewidth=3, linestyle='--')
    plt.axis('equal')
    plt.legend()

In [ ]:
plot_customer_segments(best_rotated_data)

In [ ]:
zero_y_values = np.zeros(x_values.size)

In [ ]:
reproduced_data = rotate(-optimal_angle, np.array([x_values, zero_y_values]))

In [ ]:
plt.plot(reproduced_data[0], reproduced_data[1], c='k',
label='Reproduced Data')
plt.scatter(centered_data[0], centered_data[1], c='y',
label='Original Data')
plt.axis('equal')
plt.legend()

In [ ]:
np.random.seed(1)
new_heights = np.arange(60, 78, .11)
random_fluctuations = np.random.normal(scale=10, size=new_heights.size)
new_weights = 4 * new_heights - 130 + random_fluctuations
new_centered_data = np.array([new_heights - heights.mean(),
new_weights - weights.mean()])
plt.scatter(new_centered_data[0], new_centered_data[1], c='y',
label='New Customer Data')
plt.plot(reproduced_data[0], reproduced_data[1], c='k',
label='First Principal Direction')
plt.xlabel('Centralized Height (in)')
plt.ylabel('Centralized Weight (lb)')
plt.axis('equal')
plt.legend()

In [ ]:
new_horizontal_data = rotate(optimal_angle, data=new_centered_data)
plot_customer_segments(new_horizontal_data)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_object = PCA()

In [ ]:
measurements.shape

In [ ]:
pca_transformed_data = pca_object.fit_transform(measurements)

In [ ]:
pca_transformed_data

In [ ]:
plt.scatter(pca_transformed_data[:, 0], pca_transformed_data[:, 1])
plt.axhline(0, c='black')
plt.axvline(0, c='black')
plt.axis('equal')

In [ ]:
pca_transformed_data.shape

In [ ]:
plot_customer_segments( (pca_transformed_data * np.array([1, -1])).T)

In [ ]:
percent_variance_coverage = 100 * pca_object.explained_variance_ratio_

In [ ]:
x_axis_coverage, y_axis_coverage = percent_variance_coverage

In [ ]:
f"The x-axis of our PCA output covers {x_axis_coverage:.2f}% of the total variance"

In [ ]:
first_pc = pca_object.components_[0]

In [ ]:
first_pc

In [ ]:
np.linalg.norm(first_pc)

In [ ]:
def plot_stretched_vector(v, **kwargs):
    plt.plot([-50 * v[0], 50 * v[0]], [-50 * v[1], 50 * v[1]], **kwargs)
plt.plot(reproduced_data[0], reproduced_data[1], c='k',
label='First Principal Direction')
plt.scatter(centered_data[0], centered_data[1], c='y')
plt.xlabel('Centralized Height (in)')
plt.ylabel('Centralized Weight (lb)')
plt.axis('equal')
plt.legend()

In [ ]:
measurements.shape

In [ ]:
plt.scatter(measurements[:, 0], measurements[:, 1])

In [ ]:
centered_data.shape

In [ ]:
plt.scatter(centered_data[0], centered_data[1])

In [ ]:
principal_components = pca_object.components_
for i, pc in enumerate(principal_components):
    plot_stretched_vector(pc, c='k', label='Principal Directions' if i == 0 else None)
for i, axis_vector in enumerate([np.array([0, 1]), np.array([1, 0])]):
    plot_stretched_vector(axis_vector, c='g', linestyle='-.', label='Axes' if i == 0 else None)
plt.scatter(centered_data[0], centered_data[1], c='y')
plt.xlabel('Centralized Height (in)')
plt.ylabel('Centralized Weight (lb)')
plt.legend()
plt.axis('equal')

In [ ]:
pca_object.components_

In [ ]:
measurements.shape

In [ ]:
temp=measurements @ pca_object.components_

In [ ]:
projections = pca_object.components_ @ centered_data
assert np.allclose(pca_transformed_data.T,projections)

In [ ]:
from sklearn.datasets import load_iris
flower_data = load_iris()

In [ ]:
flower_data

In [ ]:
flower_measurements = flower_data['data']

In [ ]:
flower_measurements

In [ ]:
num_flowers, num_measurements = flower_measurements.shape

In [ ]:
num_flowers

In [ ]:
num_measurements

In [ ]:
pca_object_2D = PCA(n_components=2)

In [ ]:
pca_object_2D.fit_transform(flower_measurements)

In [ ]:
transformed_data_2D = _

In [ ]:
row_count, column_count= transformed_data_2D.shape

In [ ]:
def print_2D_variance_coverage(pca_object):
    percent_var_coverages = 100 * pca_object.explained_variance_ratio_
    x_axis_coverage, y_axis_coverage = percent_var_coverages
    total_coverage = x_axis_coverage + y_axis_coverage
    print(f"The x-axis covers {x_axis_coverage:.2f}% "
    "of the total variance")
    print(f"The y-axis covers {y_axis_coverage:.2f}% "
    "of the total variance")
    print(f"Together, the 2 axes cover {total_coverage:.2f}% "
    "of the total variance")

In [ ]:
print_2D_variance_coverage(pca_object_2D)

In [ ]:
plt.scatter(transformed_data_2D[:, 0], transformed_data_2D[:, 1])

In [ ]:
def visualize_flower_data(dim_reduced_data):
    species_names = flower_data['target_names']
    for i, species in enumerate(species_names):
        species_data = np.array([dim_reduced_data[j] for j in range(dim_reduced_data.shape[0]) if flower_data['target'][j]==i]).T
        plt.scatter(species_data[0], species_data[1], label=species.title(), color=['g', 'k', 'y'][i])
    plt.legend()

In [ ]:
visualize_flower_data(dim_reduced_data=transformed_data_2D)

In [ ]:
pca_object_2D.components_

In [ ]:
def detect_setosa(flower_sample):
    centered_sample = flower_sample - pca_object_2D.mean_
    projection = pca_object_2D.components_[0] @ centered_sample
    if projection < -2:
        print("The sample could be a Setosa")
    else:
        print("The sample is not Setosa")

In [ ]:
new_flower_sample = np.array([4.8, 3.7, 1.2, 0.24])
detect_setosa(new_flower_sample)

In [ ]:
for i in range(flower_measurements.shape[1]):
    flower_measurements[:,i] /= np.linalg.norm(flower_measurements[:,i])
    transformed_data_2D = pca_object_2D.fit_transform(flower_measurements)
print_2D_variance_coverage(pca_object_2D)

In [ ]:
visualize_flower_data(transformed_data_2D)

In [ ]:
centered_data.shape

In [ ]:
centered_data @ centered_data.T

In [ ]:
conv_matrix = centered_data @ centered_data.T / centered_data.shape[1]

In [ ]:
conv_matrix

In [ ]:
for i in range(centered_data.shape[0]):
    variance = conv_matrix[i, i]
    assert round(variance, 10) == round(centered_data[i].var(), 10)

In [ ]:
def plot_vector(vector, **kwargs):
    plt.plot([0, vector[0]], [0, vector[1]], **kwargs)

In [ ]:
plot_vector(first_pc, c='y', label='First Principal Component')
product_vector = conv_matrix @ first_pc
product_vector /= np.linalg.norm(product_vector)
plot_vector(first_pc, c='k', linestyle='--', label='First Principal Component')
plt.legend()
plt.axis('equal')

In [ ]:
product_vector2 = conv_matrix @ product_vector

In [ ]:
product_vector2 /= np.linalg.norm(product_vector2)

In [ ]:
import math
cosine_similarity = product_vector @ product_vector2
angle = math.acos(cosine_similarity)

In [ ]:
f"{angle:.2f}"

In [ ]:
new_magnitude = np.linalg.norm(conv_matrix @ first_pc)

In [ ]:
new_magnitude

In [ ]:
variance = (centered_data.T @ first_pc).var()

In [ ]:
direction1_var = projections[0].var()

In [ ]:
assert round(variance, 10) == round(direction1_var, 10)
print(f"The variance along the first principal direction is approximately {variance:.1f}")

In [ ]:
centered_data.shape

In [ ]:
np.random.seed(0)
random_vector = np.random.random(size=2)

In [ ]:
random_vector

In [ ]:
random_vector /= np.linalg.norm(random_vector)

In [ ]:
random_vector

In [ ]:
product_vector = conv_matrix @ random_vector
product_vector /= np.linalg.norm(product_vector)

In [ ]:
product_vector = conv_matrix @ random_vector
product_vector /= np.linalg.norm(product_vector)
plt.plot([0, random_vector[0]], [0, random_vector[1]],
label='Random Vector')
plt.plot([0, product_vector[0]], [0, product_vector[1]], linestyle='--',
c='k', label='Normalized Product Vector')
plt.legend()
plt.axis('equal')

In [ ]:
product_vector2 = conv_matrix @ product_vector
product_vector2 /= np.linalg.norm(product_vector2)
plt.plot([0, product_vector[0]], [0, product_vector[1]], linestyle='--',
c='k', label='Normalized Product Vector')
plt.plot([0, product_vector2[0]], [0, product_vector2[1]], linestyle=':',
c='r', label='Normalized Product Vector2')
plt.legend()
plt.axis('equal')

In [ ]:
np.random.seed(0)
def power_iteration(matrix):
    random_vector = np.random.random(size=matrix.shape[0])
    random_vector /= np.linalg.norm(random_vector)
    old_rotated_vector = random_vector
    for _ in range(10):
        rotated_vector = matrix @ old_rotated_vector
        rotated_vector /= np.linalg.norm(rotated_vector)
        old_rotated_vector = rotated_vector
    eigenvector = rotated_vector
    eigenvalue = np.linalg.norm(matrix @ eigenvector)
    return eigenvector, eigenvalue

In [ ]:
eigenvector, eigenvalue = power_iteration(conv_matrix)

In [ ]:
eigenvector[1] * eigenvector[0]

In [ ]:
outer_product  = np.outer(eigenvector, eigenvector)

In [ ]:
deflated_matrix =  conv_matrix - eigenvalue * outer_product

In [ ]:
next_eigenvector, next_eigenvalue =power_iteration(deflated_matrix)

In [ ]:
np.random.seed(0)
components = np.array([eigenvector, next_eigenvector])
projection = components @ centered_data
plt.scatter(projections[0], projections[1])
plt.axis('equal')
plt.axhline(0, c='black')
plt.axvline(0, c='black')

In [ ]:
def find_top_eigenvectors(matrix, k=2):
    matrix = matrix.copy()
    eigenvectors = []
    for _ in range(k):
        eigenvector, eigenvalue = power_iteration(matrix)
        eigenvectors.append(eigenvector)
        matrix -= eigenvalue * np.outer(eigenvector, eigenvector)
    return np.array(eigenvectors)
def find_top_principal_components(centered_matrix, k=2):
    cov_matrix = centered_matrix @ centered_matrix.T / centered_matrix.shape[1]
    return find_top_eigenvectors(cov_matrix, k=k)

In [ ]:
find_top_principal_components(centered_data)

In [ ]:
conv_matrix

In [ ]:
def reduce_dimensions(data, k=2, centralized_data=True):
    data = data.T.copy()
    if centralized_data:
        for i in range(data.shape[0]):
            data[i] -= data[i].mean()
    principal_components = find_top_principal_components(data, k)
    return (principal_components @ data).T

In [ ]:
np.random.seed(0)
dim_reduced_data = reduce_dimensions(flower_measurements)
dim_reduced_data

In [ ]:
visualize_flower_data(dim_reduced_data)

In [ ]:
np.random.seed(0)
dim_reduced_data2 = reduce_dimensions(flower_measurements, centralized_data=False)

In [ ]:
visualize_flower_data(dim_reduced_data2)

In [ ]:
np.random.seed(3)
dim_reduced_data = reduce_dimensions(flower_measurements, centralized_data=False)
visualize_flower_data(dim_reduced_data)

In [ ]:
variances = [sum(data[:,i].var() for i in range(data.shape[1]))
for data in [dim_reduced_data, flower_measurements]]
dim_reduced_var, total_var = variances
percent_coverege = 100 * dim_reduced_var / total_var
print(f"Our plot covers {percent_coverege:.2f}% of the total variance")

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd_object = TruncatedSVD(n_components=2)

In [ ]:
svd_transformed_data = svd_object.fit_transform(flower_measurements)

In [ ]:
visualize_flower_data(svd_transformed_data)

In [ ]:
percent_variance_coverages = 100 * svd_object.explained_variance_ratio_
x_axis_coverage, y_axis_coverage = percent_variance_coverages
total_2d_coverage = x_axis_coverage + y_axis_coverage
print(f"Our Scikit-Learn SVD output covers {total_2d_coverage:.2f}% of "
"the total variance")